In [1]:
CELLS = [(q, r) for q in range(-3, +3 + 1) for r in range(-3, +3 + 1) if -q - r in range(-3, +3 + 1)]

In [2]:
import tensorflow as tf
import numpy as np
from keras import models, layers
from keras.models import load_model
import json
import pickle
import os
from os.path import join
import subprocess
import json
import VanGame.utils as utils
import scipy as sp
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

Using TensorFlow backend.


In [4]:
sequence = sorted([p for p in CELLS])
sequence

[(-3, 0),
 (-3, 1),
 (-3, 2),
 (-3, 3),
 (-2, -1),
 (-2, 0),
 (-2, 1),
 (-2, 2),
 (-2, 3),
 (-1, -2),
 (-1, -1),
 (-1, 0),
 (-1, 1),
 (-1, 2),
 (-1, 3),
 (0, -3),
 (0, -2),
 (0, -1),
 (0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, -3),
 (1, -2),
 (1, -1),
 (1, 0),
 (1, 1),
 (1, 2),
 (2, -3),
 (2, -2),
 (2, -1),
 (2, 0),
 (2, 1),
 (3, -3),
 (3, -2),
 (3, -1),
 (3, 0)]

In [6]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(100,activation='relu', input_shape=(11,)))
    model.add(layers.Dense(200, activation='relu'))
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse')
    return model

In [7]:
#model = build_model()

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
def bash_call(c:[str]) -> None:
    res = subprocess.check_output(c)
    ls = []
    for line in res.splitlines():
        line = line.decode('utf-8')
        ls.append(line)
    return ls

In [9]:
def join_list(l):
    res = ""
    for i in l:
        res += i + "/"
    
    res = res[:-1]
    return res

In [10]:
def load_logs():
    datapath = join(os.getcwd(),'rec/')
    van_files = bash_call(['ls', datapath])
    for i in range(len(van_files)):
        van_files[i] = 'rec/' + van_files[i]
    
    datapath = join(os.getcwd(),'hardcode_rec/')
    hardcode_file = bash_call(['ls', datapath])
    
    for i in range(len(hardcode_file)):
        hardcode_file[i] = 'hardcode_rec/' + hardcode_file[i]

    return van_files + hardcode_file

In [31]:
import math

def sigmoid(x):
    return 1/(1+math.exp(-x))

In [11]:
def PG_log(filename, condition=None, value_check=False):
    
    if (condition != None) and (condition not in filename):
        return False, False
    
    #print(filename)
    states, color = utils.load_l(filename)
    
    train = []
    value = []
    
    for s in states:
        # print(s)
        board = []
        # add board
        for cord in sequence:
            # does not include the board
            break
            board.append(s['board'][cord])
        # color 
        # board.append(color)
        board.append(s['turns'])
        board += s['utility']
        board.append(s['ev'])
        train.append(board)
        value.append(s['rew'])
        
    if value_check:
        print(value)
        
    value.reverse()
    decay = 0.7
    curr_decay = 0.7
    for i in range(1, len(value)):
        value[i] += value[i-1] * curr_decay
        curr_decay *= decay
    
    value.reverse()
    
    if value_check:
        print(value)
    
    post_value = np.array(value)
    
    post_value = np.tanh(post_value)
    
    # normalize the value
    # post_value = (post_value - post_value.mean()) / post_value.std()
    
    if value_check:
        print("\n",post_value)
    
    return train, post_value.tolist()

In [12]:
# this is only for test, don't run
t,v = PG_log('hardcode_rec/[1win]1556692620.0caobluejiba.txt', value_check=True)
#print(t)

FileNotFoundError: [Errno 2] No such file or directory: 'hardcode_rec/[1win]1556692620.0caobluejiba.txt'

In [13]:
!ls 'hardcode_rec'

'[1win]1556691305.0caoredjiba.txt'    '[1win]1556693327.0caobluejiba.txt'
'[1win]1556691368.0caogreenjiba.txt'  '[2zzz]1556691289.0caoredjiba.txt'
'[1win]1556691375.0caogreenjiba.txt'  '[2zzz]1556691294.0caoredjiba.txt'
'[1win]1556691402.0caogreenjiba.txt'  '[2zzz]1556691314.0caoredjiba.txt'
'[1win]1556691412.0caogreenjiba.txt'  '[2zzz]1556691319.0caoredjiba.txt'
'[1win]1556691437.0caogreenjiba.txt'  '[2zzz]1556691329.0caoredjiba.txt'
'[1win]1556691455.0caogreenjiba.txt'  '[2zzz]1556691349.0caoredjiba.txt'
'[1win]1556691476.0caoredjiba.txt'    '[2zzz]1556691425.0caogreenjiba.txt'
'[1win]1556691500.0caoredjiba.txt'    '[2zzz]1556691432.0caogreenjiba.txt'
'[1win]1556691506.0caoredjiba.txt'    '[2zzz]1556691449.0caogreenjiba.txt'
'[1win]1556691616.0caoredjiba.txt'    '[2zzz]1556691460.0caogreenjiba.txt'
'[1win]1556691728.0caoredjiba.txt'    '[2zzz]1556691483.0caoredjiba.txt'
'[1win]1556691738.0caoredjiba.txt'    '[2zzz]1556691493.0caoredjiba.txt'
'[1win]1556691745.0caoredjiba.txt'    '[2z

In [14]:
def reload_model():
    del model  # deletes the existing model

    # returns a compiled model
    # identical to the previous one
    model = load_model('my_model.h5')

In [15]:
def save_model():
    model.save('VanGame/my_model.h5')  # creates a HDF5 file 'my_model.h5'
    with open('VanGame/trained_model', 'wb+') as f:
        pickle.dump(model.get_weights(), f)

In [36]:
def train(epochs, save=True):
    logfs = load_logs()
    train_set = []
    target_set = []
    
    for logf in logfs:
        train, value = PG_log(logf)
        if train:
            # print(train)
            # print(len(train), len(value))
            #print(train[-1])
            train_set += train
            # the target shouldnt be calculate like this, should use bootstrapping
            target_set += value

    model.fit(np.array(train_set), target_set, epochs=epochs, batch_size=1)
    
    if save:
        print("saved")
        save_model()

In [218]:
model.predict(np.array([[-1]*11]))

array([[0.15931465]], dtype=float32)

In [17]:
# don't run this, or ask me
#save_model()
#del model

#try:
#    model = load_model('VanGame/my_model.h5')
#except:
#    print("none model found")
#    model = build_model()

Instructions for updating:
Use tf.cast instead.


In [26]:
def full_train_cycle(n, epochs=2):
    for i in range(n):
        !python3 -m referee VanGame VanGame VanGame -l logtry -v 0
    if ( len(bash_call(['ls', 'rec'])) + len(bash_call(['ls', 'hardcode_rec'])) ) > 0:
        train(epochs=epochs)
        !rm -v ./rec/*
        !rm -v ./hardcode_rec/*

In [39]:
for i in range(5000):
    print("********** starting round", i, "**********")
    full_train_cycle(1, epochs=1)

********** starting round 0 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/1
345/345 [==============================] - 1s 3ms/step - loss: 0.0049
saved
removed './rec/[1win]1556719043.0caobluejiba.txt'
removed './rec/[2zzz]1556719043.0caogreenjiba.txt'
removed './rec/[2zzz]1556719043.0caoredjiba.txt'
rm: cannot remove './hardcode_rec/*': No such file or directory
********** starting round 1 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
Epoch 1/1
524/524 [==============================] - 1s 3ms/step - loss: 0.0025
saved
removed './rec/[1win]1556719050.0caogreenjiba.txt'
removed './rec/[2zzz]1556719050.0caobluejiba.txt'
removed './rec/[2zzz]1556719050.0caoredjiba.txt'
rm: cannot remove './hardcode_rec/*': No such file or directory
********** starting round 2 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/1
411/411 [========

********** starting round 20 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Red
Epoch 1/1
256/256 [==============================] - 1s 3ms/step - loss: 0.0083
saved
removed './rec/[1win]1556719169.0caoredjiba.txt'
removed './rec/[2zzz]1556719169.0caobluejiba.txt'
removed './rec/[2zzz]1556719169.0caogreenjiba.txt'
rm: cannot remove './hardcode_rec/*': No such file or directory
********** starting round 21 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Red
Epoch 1/1
202/202 [==============================] - 1s 3ms/step - loss: 0.0092
saved
removed './rec/[1win]1556719173.0caoredjiba.txt'
removed './rec/[2zzz]1556719173.0caobluejiba.txt'
removed './rec/[2zzz]1556719173.0caogreenjiba.txt'
rm: cannot remove './hardcode_rec/*': No such file or directory
********** starting round 22 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/1
648/648 [========

********** starting round 40 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/1
252/252 [==============================] - 1s 3ms/step - loss: 0.0076
saved
removed './rec/[1win]1556719312.0caobluejiba.txt'
removed './rec/[2zzz]1556719312.0caogreenjiba.txt'
removed './rec/[2zzz]1556719312.0caoredjiba.txt'
rm: cannot remove './hardcode_rec/*': No such file or directory
********** starting round 41 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/1
129/129 [==============================] - 0s 3ms/step - loss: 0.0133
saved
removed './rec/[1win]1556719316.0caobluejiba.txt'
removed './rec/[2zzz]1556719316.0caogreenjiba.txt'
removed './rec/[2zzz]1556719316.0caoredjiba.txt'
rm: cannot remove './hardcode_rec/*': No such file or directory
********** starting round 42 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Red
Epoch 1/1
187/187 [=======

********** starting round 60 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/1
135/135 [==============================] - 0s 3ms/step - loss: 0.0161
saved
removed './rec/[1win]1556719471.0caobluejiba.txt'
removed './rec/[2zzz]1556719471.0caogreenjiba.txt'
removed './rec/[2zzz]1556719471.0caoredjiba.txt'
rm: cannot remove './hardcode_rec/*': No such file or directory
********** starting round 61 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
Epoch 1/1
632/632 [==============================] - 2s 3ms/step - loss: 0.0013
saved
removed './rec/[1win]1556719481.0caogreenjiba.txt'
removed './rec/[2zzz]1556719481.0caobluejiba.txt'
removed './rec/[2zzz]1556719481.0caoredjiba.txt'
rm: cannot remove './hardcode_rec/*': No such file or directory
********** starting round 62 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
Epoch 1/1
188/188 [====

Epoch 1/1
252/252 [==============================] - 1s 3ms/step - loss: 0.0086
saved
removed './rec/[1win]1556719601.0caobluejiba.txt'
removed './rec/[2zzz]1556719601.0caogreenjiba.txt'
removed './rec/[2zzz]1556719601.0caoredjiba.txt'
rm: cannot remove './hardcode_rec/*': No such file or directory
********** starting round 80 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Red
Epoch 1/1
310/310 [==============================] - 1s 3ms/step - loss: 0.0034
saved
removed './rec/[1win]1556719607.0caoredjiba.txt'
removed './rec/[2zzz]1556719607.0caobluejiba.txt'
removed './rec/[2zzz]1556719607.0caogreenjiba.txt'
rm: cannot remove './hardcode_rec/*': No such file or directory
********** starting round 81 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
Epoch 1/1
659/659 [==============================] - 2s 3ms/step - loss: 0.0023
saved
removed './rec/[1win]1556719617.0caogreenjiba.txt'
removed './re

...

rm: cannot remove './hardcode_rec/*': No such file or directory
********** starting round 4920 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/1
429/429 [==============================] - 1s 3ms/step - loss: 0.0050
saved
removed './rec/[1win]1556754549.0caobluejiba.txt'
removed './rec/[2zzz]1556754549.0caogreenjiba.txt'
removed './rec/[2zzz]1556754549.0caoredjiba.txt'
rm: cannot remove './hardcode_rec/*': No such file or directory
********** starting round 4921 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Red
Epoch 1/1
712/712 [==============================] - 2s 3ms/step - loss: 0.0026
saved
removed './rec/[1win]1556754561.0caoredjiba.txt'
removed './rec/[2zzz]1556754561.0caobluejiba.txt'
removed './rec/[2zzz]1556754561.0caogreenjiba.txt'
rm: cannot remove './hardcode_rec/*': No such file or directory
********** starting round 4922 **********
* NOTE: unable to measure memory usage on

* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/1
432/432 [==============================] - 1s 3ms/step - loss: 0.0036
saved
removed './rec/[1win]1556754691.0caobluejiba.txt'
removed './rec/[2zzz]1556754691.0caogreenjiba.txt'
removed './rec/[2zzz]1556754691.0caoredjiba.txt'
rm: cannot remove './hardcode_rec/*': No such file or directory
********** starting round 4940 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
Epoch 1/1
422/422 [==============================] - 1s 3ms/step - loss: 0.0025
saved
removed './rec/[1win]1556754699.0caogreenjiba.txt'
removed './rec/[2zzz]1556754699.0caobluejiba.txt'
removed './rec/[2zzz]1556754699.0caoredjiba.txt'
rm: cannot remove './hardcode_rec/*': No such file or directory
********** starting round 4941 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/1
414/414 [==============================] - 1s 3ms/

********** starting round 4959 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/1
585/585 [==============================] - 2s 3ms/step - loss: 0.0018
saved
removed './rec/[1win]1556754819.0caobluejiba.txt'
removed './rec/[2zzz]1556754819.0caogreenjiba.txt'
removed './rec/[2zzz]1556754819.0caoredjiba.txt'
rm: cannot remove './hardcode_rec/*': No such file or directory
********** starting round 4960 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
Epoch 1/1
257/257 [==============================] - 1s 3ms/step - loss: 0.0041
saved
removed './rec/[1win]1556754825.0caogreenjiba.txt'
removed './rec/[2zzz]1556754825.0caobluejiba.txt'
removed './rec/[2zzz]1556754825.0caoredjiba.txt'
rm: cannot remove './hardcode_rec/*': No such file or directory
********** starting round 4961 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
Epoch 1/1
116/116

* winner: Red
Epoch 1/1
367/367 [==============================] - 1s 3ms/step - loss: 0.0058
saved
removed './rec/[1win]1556754946.0caoredjiba.txt'
removed './rec/[2zzz]1556754946.0caogreenjiba.txt'
removed './rec/[2zzz]1556754947.0caobluejiba.txt'
rm: cannot remove './hardcode_rec/*': No such file or directory
********** starting round 4979 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/1
369/369 [==============================] - 1s 3ms/step - loss: 0.0032
saved
removed './rec/[1win]1556754953.0caobluejiba.txt'
removed './rec/[2zzz]1556754953.0caogreenjiba.txt'
removed './rec/[2zzz]1556754953.0caoredjiba.txt'
rm: cannot remove './hardcode_rec/*': No such file or directory
********** starting round 4980 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/1
330/330 [==============================] - 1s 3ms/step - loss: 0.0064
saved
removed './rec/[1win]1556754960.0caobluejiba.t

* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/1
309/309 [==============================] - 1s 3ms/step - loss: 0.0068
saved
removed './rec/[1win]1556755097.0caobluejiba.txt'
removed './rec/[2zzz]1556755097.0caogreenjiba.txt'
removed './rec/[2zzz]1556755097.0caoredjiba.txt'
rm: cannot remove './hardcode_rec/*': No such file or directory
********** starting round 4999 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
Epoch 1/1
200/200 [==============================] - 1s 3ms/step - loss: 0.0095
saved
removed './rec/[1win]1556755101.0caogreenjiba.txt'
removed './rec/[2zzz]1556755101.0caobluejiba.txt'
removed './rec/[2zzz]1556755101.0caoredjiba.txt'
rm: cannot remove './hardcode_rec/*': No such file or directory


In [35]:
for i in range(2):
    print("********** starting round", i, "**********")
    full_train_cycle(2, epochs=1)

********** starting round 0 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* draw detected: maximum number of turns reached.
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
76 76
173 173
75 75
76 76
173 173
173 173
Epoch 1/1
746/746 [==============================] - 2s 3ms/step - loss: 0.0049
saved
removed './rec/[1win]1556703582.0caogreenjiba.txt'
removed './rec/[1win]1556703634.0caobluejiba.txt'
removed './rec/[2zzz]1556703582.0caobluejiba.txt'
removed './rec/[2zzz]1556703582.0caoredjiba.txt'
removed './rec/[2zzz]1556703634.0caogreenjiba.txt'
removed './rec/[2zzz]1556703634.0caoredjiba.txt'
rm: cannot remove './hardcode_rec/*': No such file or directory
********** starting round 1 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* draw detected: maximum number of turns reached.
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
161 161
161 161
161 

# CNN attamp

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
#create model
md = Sequential()
#add model layers
md.add(Conv2D(1, kernel_size=3, activation='relu', input_shape=(28,28,1), use_bias=False))
#md.add(layers.MaxPooling2D(2,2))
#md.add(Conv2D(64, kernel_size=3, activation='relu', use_bias=False))
#md.add(layers.MaxPooling2D(2,2))
#md.add(Conv2D(64, kernel_size=3, activation='relu', use_bias=False))
md.add(Flatten())
md.compile(optimizer='adam', loss='mse')

In [ ]:
for i in md.get_weights():
    print(i.shape)

In [ ]:
md.summary()

In [ ]:
md.predict(np.array([[[[1]*28]*28]]).reshape(1,28,28,1))

In [ ]:
asdf = md.get_weights()[0].reshape(3,3)

In [ ]:
wds = md.get_weights()

In [ ]:
convp = wds[0]

In [ ]:
convp.shape

In [ ]:
c = convp.reshape(1, 3, 3, 1)

In [ ]:
d = c[0].reshape(3,3)

In [ ]:
import scipy.signal as signal

In [ ]:
a = np.array([[[1]*28]*28]).reshape(28,28)

In [ ]:
e = signal.convolve2d(a, asdf, mode='valid')

In [ ]:
e

In [ ]:
signal.convolve(e, )